<a href="https://www.kaggle.com/code/ibkya12/map-0-97-leashmp-competition?scriptVersionId=180384332" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 37.4 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 57.4 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,145017544,O=C(Nc1c(Cl)cc(Cl)nc1C(=O)O)OCC1c2ccccc2-c2ccc...,CCON(C)C(=O)CN.Cl,Nc1ncc(F)cc1[N+](=O)[O-],CCON(C)C(=O)CNc1nc(Nc2ncc(F)cc2[N+](=O)[O-])nc...,HSA,0
1,282866642,O=C(O)[C@@H]1CSCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cn1ncc2cc(N)ccc21,Cc1cc(CN)cc(Cl)n1.Cl.Cl,Cc1cc(CNc2nc(Nc3ccc4c(cnn4C)c3)nc(N3CSC[C@H]3C...,sEH,0
2,285466885,O=C(O)[C@@H]1Cc2ccccc2CN1C(=O)OCC1c2ccccc2-c2c...,CC(=O)c1cccc(N)c1,Cc1cc(N)ncc1[N+](=O)[O-],CC(=O)c1cccc(Nc2nc(Nc3cc(C)c([N+](=O)[O-])cn3)...,HSA,0
3,2011596,C#CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Nc1ccc(N2CCC(N3CCOCC3)CC2)cc1,NCCOc1cccnc1,C#CC[C@@H](Nc1nc(NCCOc2cccnc2)nc(Nc2ccc(N3CCC(...,BRD4,0
4,254433729,O=C(O)C[C@@H](Cc1cccs1)NC(=O)OCC1c2ccccc2-c2cc...,Cl.NCCC1CCCC1(F)F,CCOC(=O)c1csc(N)n1,CCOC(=O)c1csc(Nc2nc(NCCC3CCCC3(F)F)nc(N[C@@H](...,BRD4,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [7]:
from xgboost import XGBClassifier

In [8]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


In [9]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.052343
0:	learn: 0.6743255	total: 106ms	remaining: 1m 45s
1:	learn: 0.6565343	total: 147ms	remaining: 1m 13s
2:	learn: 0.6422303	total: 190ms	remaining: 1m 3s
3:	learn: 0.6268796	total: 232ms	remaining: 57.8s
4:	learn: 0.6145348	total: 272ms	remaining: 54.1s
5:	learn: 0.6046016	total: 317ms	remaining: 52.5s
6:	learn: 0.5934765	total: 358ms	remaining: 50.8s
7:	learn: 0.5847465	total: 399ms	remaining: 49.5s
8:	learn: 0.5758717	total: 441ms	remaining: 48.5s
9:	learn: 0.5673930	total: 494ms	remaining: 48.9s
10:	learn: 0.5616698	total: 531ms	remaining: 47.8s
11:	learn: 0.5555882	total: 571ms	remaining: 47s
12:	learn: 0.5500864	total: 621ms	remaining: 47.2s
13:	learn: 0.5450332	total: 675ms	remaining: 47.5s
14:	learn: 0.5389158	total: 726ms	remaining: 47.7s
15:	learn: 0.5354133	total: 765ms	remaining: 47.1s
16:	learn: 0.5299987	total: 808ms	remaining: 46.7s
17:	learn: 0.5247989	total: 851ms	remaining: 46.5s
18:	learn: 0.5210586	total: 890ms	remaining: 46s
19:	learn: 0.

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [10]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
